#### Problem statement: 

During these COVID times, indoor places, such as banks, restaurants and stores are legally mandated to ensure that visitors wear masks for their own safety and stop the spread of virus. But the purpose of wearking masks gets defeated when there is a person meeting everyone to see if they are wearing masks. In such a scenario, a real-time face mask detection system comes in picture. It is not a new concept in machine learning. Computer vision has been used in image recognition and face detection systems. A common example is face detection in phone unlocking mechanisms. 

The goal of this face mask detection system is to create an image recognition system which understands how image classification works, and it should work with great accuracy so that our model can be applied in the realtime situations. It will work by recognizing the boundaries of the face and predicting whether or not you are wearing a face mask in real-time. We would utilize VideoCapture function in the OpenCV library in Python. The Cascade classifier, designed by OpenCV, can be used to detect the frontal face in live video via detectMultiScale. We will use a while loop to continue capturing images from the webcam.

Based on the performance and accuracy of our model, the result of the binary classifier will be indicated by showing a green rectangle superimposed around the section of the face indicating that the person at the camera is wearing a mask, or a red rectangle indicating that the person on camera is not wearing a mask.

Let us build our model based on the above steps:

### Libraries

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense

In [2]:
#Have used 3 dense layers with 50, 35 and 2 neurons respectively. 
#The dense network produces the probability of the binary classification of no mask = 1 and mask = 0

cnn = Sequential([Conv2D(filters=100, kernel_size=(3,3), 
                    activation='relu'),
                    MaxPooling2D(pool_size=(2,2)),
                    Conv2D(filters=100, kernel_size=(3,3), 
                    activation='relu'),
                    MaxPooling2D(pool_size=(2,2)),
                    Flatten(),
                    Dropout(0.5),
                    Dense(50),
                    Dense(35),
                    Dense(2)])

cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [3]:
import numpy as np

In [4]:
import cv2 as cv

In [5]:
labels_dict={0:'No mask',1:'Mask'}
color_dict={0:(0,0,255),1:(0,255,0)}
imgsize = 4 #set image resize
camera = cv.VideoCapture(0) # Turn on camera
# Identify frontal face
classifier = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')

2022-04-22 11:14:51.710 python[10326:1982519] mac-virtualcam(DAL): PlugInMain version=1.3.0
2022-04-22 11:14:51.710 python[10326:1982519] mac-virtualcam(DAL): HardwarePlugIn_QueryInterface 
2022-04-22 11:14:51.710 python[10326:1982519] mac-virtualcam(DAL): HardwarePlugIn_Release sRefCount now = 0
2022-04-22 11:14:51.710 python[10326:1982519] mac-virtualcam(DAL): HardwarePlugIn_InitializeWithObjectID self=0x142e49478
2022-04-22 11:14:51.711 python[10326:1982519] mac-virtualcam(DAL): HardwarePlugIn_ObjectSetPropertyData OBSDALDevice(33) kCMIOObjectPropertyListenerAdded self=0x142e49478 data(int)=1684629094
2022-04-22 11:14:51.711 python[10326:1982519] mac-virtualcam(DAL): HardwarePlugIn_ObjectSetPropertyData OBSDALDevice(33) kCMIOObjectPropertyListenerAdded self=0x142e49478 data(int)=1869180523
2022-04-22 11:14:51.711 python[10326:1982519] mac-virtualcam(DAL): HardwarePlugIn_ObjectSetPropertyData OBSDALDevice(33) kCMIOObjectPropertyListenerAdded self=0x142e49478 data(int)=1885762592
2022

In [ ]:
while True:
    (rval, im) = camera.read()
    im=cv.flip(im,1,1) #mirrow the image
    imgs = cv.resize(im, (im.shape[1] // imgsize, im.shape[0] //
    imgsize))
    face_rec = classifier.detectMultiScale(imgs) 
    for i in face_rec: # Overlay rectangle on face
        (x, y, l, w) = [v * imgsize for v in i] 
        face_img = im[y:y+w, x:x+l]
        resized=cv.resize(face_img,(150,150))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=cnn.predict(reshaped)
        label=np.argmax(result,axis=1)[0]
        cv.rectangle(im,(x,y),(x+l,y+w),color_dict[label],2)
        cv.rectangle(im,(x,y-40),(x+l,y),color_dict[label],-1)
        cv.putText(im, labels_dict[label], (x, y-
        10),cv.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    cv.imshow('LIVE',im)
    key = cv.waitKey(10)
    # stop loop by ESC
    if key == 27: # The Esc key
        break
camera.release()
cv.destroyAllWindows()

2022-04-22 11:14:54.160344: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-22 11:14:54.160785: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 4. Tune using inter_op_parallelism_threads for best performance.
